# 3.1 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot
* Chain-Of-Thought
* [LangChain Document Loader](https://python.langchain.com/v0.2/docs/how_to/document_loader_pdf/)

In [1]:
import asyncio
import os

from utils.pdf2base64 import pdf2base64
from dotenv import load_dotenv, find_dotenv

from langchain_openai import ChatOpenAI
from langchain.schema.messages import AIMessage, HumanMessage
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

load_dotenv(find_dotenv())

model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4o",
    model_kwargs={"top_p": 0, "seed": 42},
    temperature=0,
)

In [77]:
# A) OCR is done via RapidOCR --> CNN/RNN != Transformer --> Emebddings + VectorStore
pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf"
loader = PyMuPDFLoader(pdf_path, extract_images=True)
data = loader.load()
data[0]

Document(page_content='19.06.2000\nOviS\nG\n9833\nSCHRIFTLICHER TEIL (Teil B)\nBEBAUUNGSPLAN "GEWERBEGEBIET HIMMELREICH"\nStadt LaIChInGen, GemarKUNG FeldSTeTfen, AlB-DoNaU-KreiS\nLageplan M 1: 500\nFur die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:\nBaugesetzbuch (BauGB)\nin der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).\nBaunutzungsverordnung (BauNvo)\n g   5        \nam 22.04.1993 (BGBl. 1. S. 466).\nPianzeichenverordnung 1990 (PlanzV 90)\nin der Fassung der Bekanntmachung vom 18.12.1990 (BGBl. I. S. 58).\nLandesbauordnung (LBO)\nin der Fassung der Bekanntmachung vom 08.08.1995 (GB1. S. 617).\nBisherige Festsetzungen:\nMit in Kraft treten dieses Bebauungsplanes treten im Geltungsbereich alle bisherigen\ngemeindlichen bauplanungsrechtlichen und bauordnungsrechtichen Festsetzungen auBer Kraft.', metadata={'source': '../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf', 'file_path': '

In [8]:
# B) OCR via GPT-4o --> Emebddings + VectorStore
pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf"

def pdf2prompts(pdf_path):
    prompts = []
    while True:
        try:
            page_number = len(prompts)
            base64_page=pdf2base64(pdf_path, page_number)
            image_url_prompt = {"type": "image_url","image_url": {"url": f"data:image/jpeg;base64,{base64_page[0]}","detail": "high"}}
            prompts.append(image_url_prompt)
        except Exception:
            break
    return prompts

async def page2text(page_prompt):
    msg = model.invoke([
        AIMessage(
            content=[
                {
                    "type": "text",
                    "text": "Du bist ein Assistent um Textinformationen aus dem schriftlichen Teil eines Bebauungsplans zu extrahieren."
                },
            ]
        ),
        HumanMessage(
            content=[
            {
                "type": "text",
                "text": 'Extrahiere den kompletten Textinhalt. Ausgabe im XML-Format'
            },
            page_prompt
            ]
        ),
    ])
    return msg.content

prompts = pdf2prompts(pdf_path)
data_loaders = list(map(lambda prompt: page2text(prompt), prompts))
data = await asyncio.gather(*data_loaders)
print(data[0])

In [22]:
# B) QA directly in the chat (e.g. summarization, extraction, etc.)
pdf_prompts = pdf2prompts(pdf_path)
msg = model.invoke([
        AIMessage(
            content=[
                {
                    "type": "text",
                    "text": "Du bist ein Assistent um Informationen aus einem Bebauungsplan zu extrahieren."
                },
            ]
        ),
        HumanMessage(
            content=[
                {
                    "type": "text",
                    "text": 'Fasse relevante Informationen zu den folgenden Themen zusammen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, Dachform und Dachneigung.'
                },
                *pdf_prompts,
            ]
        ),
    ])
print(msg.content)

Hier sind die relevanten Informationen aus dem Bebauungsplan "Gewerbegebiet Himmelreich" zusammengefasst:

### Art der baulichen Nutzung
- **Eingeschränktes Gewerbegebiet (GEE)** gemäß § 8 BauNVO i.V.m. § 1 (4) BauNVO:
  - Zulässig sind Gewerbebetriebe, die das Wohnen nicht wesentlich stören, einschließlich Einzelhandelsbetriebe mit bestimmten Sortimentsbeschränkungen (z.B. Kfz, Motorräder, Fahrräder, Brennstoffe).
  - Nicht zulässig sind Vergnügungsstätten und Einzelhandelsbetriebe mit nicht innenstadtrelevanten Sortimenten.
- **Gewerbegebiet (GE)** gemäß § 8 BauNVO:
  - Zulässig sind Gewerbebetriebe aller Art, einschließlich Einzelhandelsbetriebe mit bestimmten Sortimentsbeschränkungen.
  - Nicht zulässig sind Vergnügungsstätten und Einzelhandelsbetriebe mit nicht innenstadtrelevanten Sortimenten.

### Maß der baulichen Nutzung
- **Grundflächenzahl (GRZ)** gemäß § 19 BauNVO: Siehe Einträge im Lageplan.
- **Geschossflächenzahl (GFZ)** gemäß § 20 BauNVO: Siehe Einträge im Lageplan.
- *